In [15]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [17]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF3'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = np.subtract(data,rf)

actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.740000,-0.490000,-1.680000,2.020000,-0.960000,-0.500000,-1.300000,0.290000,0.530000,-1.880000,-0.390000
196308,4.610000,6.290000,5.940000,3.670000,4.880000,4.030000,6.140000,9.300000,3.940000,5.230000,5.060000
196309,-1.943333,-0.503333,-1.023333,-3.903333,-0.133333,2.096667,0.696667,-4.323333,-2.763333,-3.423333,-1.563333
196310,2.390000,9.450000,2.310000,-0.590000,8.020000,3.130000,0.210000,3.110000,-0.940000,1.110000,2.550000
196311,-1.406667,-5.116667,0.023333,-1.426667,-0.566667,3.883333,-1.506667,-1.926667,-1.296667,-0.046667,-0.856667


In [18]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [19]:
new_returns_data = returns_data[119:len(returns_data)]
returns_monthly = new_returns_data
returns_mean = returns_monthly.mean()
cov_matrix = returns_monthly.cov()
    
# udregner profit, volatilitet og sharpe
tl = np.dot(np.linalg.inv(cov_matrix), returns_mean)
nl = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), returns_mean))
weights = tl / nl
    
# udregner afkast for måned T + 1 med de udregnede optimale vægte
returns = np.dot(weights, returns_mean)
variance = np.dot(weights, np.dot(cov_matrix, weights))
sharpe = returns / np.sqrt(variance)

# indsætter overstående udregninger i vektorerne
sharpe_ratio.append(sharpe)
port_returns.append(returns)
port_volatility.append(variance)
stock_weights.append(weights)

In [20]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Variance': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [21]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [22]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Variance', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Variance,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,1.114692,27.312537,0.213292,1.50448,-0.059207,0.132286,0.898661,0.956964,0.667894,0.087992,0.125279,0.088776,0.89084,-4.293964


In [23]:
df['Sharpe Ratio']

0    0.213292
Name: Sharpe Ratio, dtype: float64